# Stage 3: dataset cleaning and train-test split
In this stage of the project, we are going to create data preparation pipeline. The pipeline is a sequential algorithm for data preparation, missing values cleaning/imputing, features scaling, and data processing.

Finally, we are going to split the initial dataset into training data, and testing data. Training data is used for machine-learning model fitting, and test data is used for validation of the model.

We prepare the train/test dataset splitting code, which preserves the balance of the dataset. Correct train/test splitting is very important because machine learning algorithms require balanced distribution of target metrics. 

The house prices in train/test datasets must have similar distribution parameters. Otherwise, if we don't create balanced train/test datasets, it will lead to poor model results.

In [1]:
%pylab inline

import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = pd.read_csv('data/house_prices.csv').set_index('Id')

# Pipeline creation

### Defining custom pipeline steps

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
import re


class DataFrameSelector(BaseEstimator, TransformerMixin):
    """
    This pipeline step selects only defined attribute names from the dataset
    """
    def __init__(self, attribute_names=None):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.attribute_names is None:
            return X
        else:
            return X.copy()[self.attribute_names]



class DataFrameConvertCategorical(BaseEstimator, TransformerMixin):
    """
    This is a custom data imputing pipeline step:
        converts string categorical values to numbers + applies one-hot-encoding
    """
    def __init__(self, attribute_dict, out_dataframe=False):
        self.attribute_dict = attribute_dict
        self.out_dataframe = out_dataframe

        if not self.attribute_dict:
            raise ValueError("Empty 'attribute_dict' for DataFrameSelector")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame), 'DataFrame expected'

        result = pd.DataFrame([], index=X.index)

        for col_name, cat_names in self.attribute_dict.items():
            cat_series = X[col_name].copy()

            # Replace NaN values
            cat_series.fillna('NA', inplace=True)

            # Set cat series as string
            cat_series = cat_series.astype(str)

            for cat in cat_names:
                new_col = f'{col_name}_{cat}'

                # Clean column_names
                new_col = re.sub('[^0-9a-zA-Z]+', '_', new_col)

                # Calculate categories values
                cat_val = cat_series == cat

                result[new_col] = cat_val.astype(np.uint8)

        self.cols_ = list(result.columns)

        if self.out_dataframe:
            return result
        else:
            return result.values

### Filtering features (defining which features to keep)

In [14]:
CATEGORICAL_ATTRS = {
'MSSubClass'        : ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190'], # Identifies the type of dwelling involved in the sale.
'MSZoning'          : ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'], # Identifies the general zoning classification of the sale.
'Street'            : ['Grvl', 'Pave'], # Type of road access to property
'Alley'             : ['Grvl', 'Pave', 'NA'], # Type of alley access to property
'LotShape'          : ['Reg', 'IR1', 'IR2', 'IR3'], # General shape of property
'LandContour'       : ['Lvl', 'Bnk', 'HLS', 'Low'], # Flatness of the property
'Utilities'         : ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], # Type of utilities available
'LotConfig'         : ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'], # Lot configuration
'LandSlope'         : ['Gtl', 'Mod', 'Sev'], # Slope of property
'Neighborhood'      : ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'Names', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'], # Physical locations within Ames city limits
'Condition1'        : ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], # Proximity to various conditions
'Condition2'        : ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], # Proximity to various conditions (if more than one is present)
'BldgType'          : ['1Fam', '2FmCon', 'Duplx', 'TwnhsE', 'TwnhsI'], # Type of dwelling
'HouseStyle'        : ['1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl'], # Style of dwelling
'OverallQual'       : ['10', '9', '8', '7', '6', '5', '4', '3', '2', '1'], # Rates the overall material and finish of the house
'OverallCond'       : ['10', '9', '8', '7', '6', '5', '4', '3', '2', '1'], # Rates the overall condition of the house
'RoofStyle'         : ['Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'], # Type of roof
'RoofMatl'          : ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'], # Roof material
'Exterior1st'       : ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd', 'WdShing'], # Exterior covering on house
'Exterior2nd'       : ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd', 'WdShing'], # Exterior covering on house (if more than one material)
'MasVnrType'        : ['BrkCmn', 'BrkFace', 'CBlock', 'None', 'Stone'], # Masonry veneer type
'ExterQual'         : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], # Evaluates the quality of the material on the exterior
'ExterCond'         : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], # Evaluates the present condition of the material on the exterior
'Foundation'        : ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], # Type of foundation
'BsmtQual'          : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], # Evaluates the height of the basement
'BsmtCond'          : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], # Evaluates the general condition of the basement
'BsmtExposure'      : ['Gd', 'Av', 'Mn', 'No', 'NA'], # Refers to walkout or garden level walls
'BsmtFinType1'      : ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'], # Rating of basement finished area
'BsmtFinType2'      : ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'], # Rating of basement finished area (if multiple types)
'Heating'           : ['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'], # Type of heating
'HeatingQC'         : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], # Heating quality and condition
'CentralAir'        : ['N', 'Y'], # Central air conditioning
'Electrical'        : ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'], # Electrical system
'KitchenQual'       : ['Ex', 'Gd', 'TA', 'Fa', 'Po'], # Kitchen quality
'Functional'        : ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'], # Home functionality (Assume typical unless deductions are warranted)
'FireplaceQu'       : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], # Fireplace quality
'GarageType'        : ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd', 'NA'], # Garage location
'GarageFinish'      : ['Fin', 'RFn', 'Unf', 'NA'], # Interior finish of the garage
'GarageQual'        : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], # Garage quality
'GarageCond'        : ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], # Garage condition
'PavedDrive'        : ['Y', 'P', 'N'], # Paved driveway
'PoolQC'            : ['Ex', 'Gd', 'TA', 'Fa', 'NA'], # Pool quality
'Fence'             : ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'], # Fence quality
'MiscFeature'       : ['Elev', 'Gar2', 'Othr', 'Shed', 'TenC', 'NA'], # Miscellaneous feature not covered in other categories
'SaleType'          : ['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth'], # Type of sale
'SaleCondition'     : ['Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial'], # Condition of sale
}

NUMERICAL_ATTRS = [
'LotFrontage',       # Linear feet of street connected to property
'LotArea',           # Lot size in square feet
'YearBuilt',         # Original construction date
'YearRemodAdd',      # Remodel date (same as construction date if no remodeling or additions)
'MasVnrArea',        # Masonry veneer area in square feet
'BsmtFinSF1',        # Type 1 finished square feet
'BsmtFinSF2',        # Type 2 finished square feet
'BsmtUnfSF',         # Unfinished square feet of basement area
'TotalBsmtSF',       # Total square feet of basement area
'1stFlrSF',          # First Floor square feet
'2ndFlrSF',          # Second floor square feet
'LowQualFinSF',      # Low quality finished square feet (all floors)
'GrLivArea',         # Above grade (ground) living area square feet
'BsmtFullBath',      # Basement full bathrooms
'BsmtHalfBath',      # Basement half bathrooms
'FullBath',          # Full bathrooms above grade
'HalfBath',          # Half baths above grade
'BedroomAbvGr',      # Bedrooms above grade (does NOT include basement bedrooms)
'KitchenAbvGr',      # Kitchens above grade
'TotRmsAbvGrd',      # Total rooms above grade (does not include bathrooms)
'Fireplaces',        # Number of fireplaces
'GarageYrBlt',       # Year garage was built
'GarageCars',        # Size of garage in car capacity
'GarageArea',        # Size of garage in square feet
'WoodDeckSF',        # Wood deck area in square feet
'OpenPorchSF',       # Open porch area in square feet
'EnclosedPorch',     # Enclosed porch area in square feet
'3SsnPorch',         # Three season porch area in square feet
'ScreenPorch',       # Screen porch area in square feet
'PoolArea',          # Pool area in square feet
'MiscVal',           # $Value of miscellaneous feature
'MoSold',            # Month Sold (MM)
'YrSold',            # Year Sold (YYYY)
        
]


### Defining data cleaning pipeline

In [74]:
# This module is not included into the sample project
from pipelines import *

from sklearn.pipeline import make_pipeline

#
# Numeric features pipeline definition
#
numeric_pipeline = make_pipeline(    
    # Select numeric features from the dataset
    DataFrameSelector(NUMERICAL_ATTRS),
    
    # Impute missing values (replace NaN by mean())
    DataFrameFunctionTransformer(func = np.mean, impute=True),
        
    # Transform features values 
    DataFrameFunctionTransformer(func = np.log1p)         
)

#
# Categorical features pipeline definition
#
categorical_pipeline = make_pipeline(
    # Convert categorical string values to numbers + apply one-hot-encoding
    DataFrameConvertCategorical(CATEGORICAL_ATTRS, out_dataframe=True),
)


data_processing_pipeline = DataFrameFeatureUnion([numeric_pipeline, categorical_pipeline])

## Data cleaning / preparation pipeline example

### Cleaned and processed dataset

In [83]:
processed_dataset = data_processing_pipeline.fit_transform(dataset)
processed_dataset.head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_ConLI,SaleType_ConLD,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,4.189655,9.042040,7.602900,7.602900,5.283204,6.561031,0.0,5.017280,6.753438,6.753438,...,0,0,0,0,1,0,0,0,0,0
2,4.394449,9.169623,7.589336,7.589336,0.000000,6.886532,0.0,5.652489,7.141245,7.141245,...,0,0,0,0,1,0,0,0,0,0
3,4.234107,9.328212,7.601902,7.602401,5.093750,6.188264,0.0,6.075346,6.825460,6.825460,...,0,0,0,0,1,0,0,0,0,0
4,4.110874,9.164401,7.557995,7.586296,0.000000,5.379897,0.0,6.293419,6.629363,6.869014,...,0,0,0,0,0,1,0,0,0,0
5,4.442651,9.565284,7.601402,7.601402,5.860786,6.486161,0.0,6.196444,7.044033,7.044033,...,0,0,0,0,1,0,0,0,0,0


### Processed dataset overview

#### Highlights
* Categorical features converted to binary columns with names, like LotShape_Reg, LotShape_IR1, LotShape_IR2
* Missind data was imputed and cleaned
* Numerical features transformed using log(1 + x) function

**Later** we will move pipeline code to 'pipeline.py' module for the future use.

In [84]:
# This module is not included into the sample project
from vizualization import *

print_features_info(processed_dataset)

Feature                       dtype       #missing  %missing   FeatureType                   

LotFrontage                   float64            0       0.0%   Numeric   
LotArea                       float64            0       0.0%   Numeric   
YearBuilt                     float64            0       0.0%   Numeric   
YearRemodAdd                  float64            0       0.0%   Numeric   
MasVnrArea                    float64            0       0.0%   Numeric   
BsmtFinSF1                    float64            0       0.0%   Numeric   
BsmtFinSF2                    float64            0       0.0%   Numeric   
BsmtUnfSF                     float64            0       0.0%   Numeric   
TotalBsmtSF                   float64            0       0.0%   Numeric   
1stFlrSF                      float64            0       0.0%   Numeric   
2ndFlrSF                      float64            0       0.0%   Numeric   
LowQualFinSF                  float64            0       0.0%   Numeric   
GrLiv

Neighborhood_Veenker          uint8              0       0.0%   Categorical (2)
Condition1_Artery             uint8              0       0.0%   Categorical (2)
Condition1_Feedr              uint8              0       0.0%   Categorical (2)
Condition1_Norm               uint8              0       0.0%   Categorical (2)
Condition1_RRNn               uint8              0       0.0%   Categorical (2)
Condition1_RRAn               uint8              0       0.0%   Categorical (2)
Condition1_PosN               uint8              0       0.0%   Categorical (2)
Condition1_PosA               uint8              0       0.0%   Categorical (2)
Condition1_RRNe               uint8              0       0.0%   Categorical (2)
Condition1_RRAe               uint8              0       0.0%   Categorical (2)
Condition2_Artery             uint8              0       0.0%   Categorical (2)
Condition2_Feedr              uint8              0       0.0%   Categorical (2)
Condition2_Norm               uint8     

Foundation_CBlock             uint8              0       0.0%   Categorical (2)
Foundation_PConc              uint8              0       0.0%   Categorical (2)
Foundation_Slab               uint8              0       0.0%   Categorical (2)
Foundation_Stone              uint8              0       0.0%   Categorical (2)
Foundation_Wood               uint8              0       0.0%   Categorical (2)
BsmtQual_Ex                   uint8              0       0.0%   Categorical (2)
BsmtQual_Gd                   uint8              0       0.0%   Categorical (2)
BsmtQual_TA                   uint8              0       0.0%   Categorical (2)
BsmtQual_Fa                   uint8              0       0.0%   Categorical (2)
BsmtQual_Po                   uint8              0       0.0%   Categorical (1)
BsmtQual_NA                   uint8              0       0.0%   Categorical (2)
BsmtCond_Ex                   uint8              0       0.0%   Categorical (1)
BsmtCond_Gd                   uint8     

SaleType_ConLD                uint8              0       0.0%   Categorical (2)
SaleType_Oth                  uint8              0       0.0%   Categorical (2)
SaleCondition_Normal          uint8              0       0.0%   Categorical (2)
SaleCondition_Abnorml         uint8              0       0.0%   Categorical (2)
SaleCondition_AdjLand         uint8              0       0.0%   Categorical (2)
SaleCondition_Alloca          uint8              0       0.0%   Categorical (2)
SaleCondition_Family          uint8              0       0.0%   Categorical (2)
SaleCondition_Partial         uint8              0       0.0%   Categorical (2)


## Train/test split

Train/test split is essential for the future stability of the machine-learning model we are going to use. Because of this, it is critical to preserve class balance for classification tasks and use shuffled selection for regression tasks.

In [87]:
from sklearn.model_selection import train_test_split

In [99]:
y = dataset['SalePrice']
X = processed_dataset



X_train, X_test, y_train, y_test = train_test_split(X, y, 
        test_size=0.25,  # Proportion of the test dataset
        random_state=64, # Fix the state of random generator, to get repeatitive results each time
        shuffle=True,    # Randomly pick test dataset to maintain original distribution                                                   
        )


In [102]:
print(f"Train dataset length: {len(X_train):<5} ({len(X_train)/len(X)*100:<2}%)")
print(f"Test  dataset length: {len(X_test):<5} ({len(X_test)/len(X)*100:<2}%)")

Train dataset length: 1095  (75.0%)
Test  dataset length: 365   (25.0%)
